In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import SubprocVecEnv
from stable_baselines3.common.callbacks import CheckpointCallback

from lib.environment.attacker.environment import Environment

import os
from datetime import datetime

In [ ]:
task_training_name = "attacker"
algorithm_name = "PPO"

In [ ]:
def create_env():
    def _init():
        return Environment()
    return _init

In [ ]:
def create_folder_if_not_exists(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

In [ ]:
def get_task_models_path():
    current_datetime = datetime.now()

    year = current_datetime.year
    month = current_datetime.month
    day = current_datetime.day
    hour = current_datetime.hour
    minute = current_datetime.minute
    second = current_datetime.second

    datetime_name = f"{year}_{month}_{day}_{hour}_{minute}_{second}"

    return f"models/{task_training_name}/{algorithm_name}/{datetime_name}"

In [ ]:
save_path = get_task_models_path()

create_folder_if_not_exists(save_path)

In [ ]:
num_threads = 60
model_filename_prefix = f"{algorithm_name}_model"

In [ ]:
env = SubprocVecEnv([create_env() for i in range(num_threads)])

n_actions = env.action_space.shape[-1]
n_actions

In [ ]:
gae_lambda = 0.95
gamma = 0.99
learning_rate = 0.0004
clip_range = 0.2
policy = "MlpPolicy"
batch_size = 128

In [ ]:
load_model = True
loaded_model_path = "models/attacker/PPO/2024_6_4_17_29_21/PPO_model"

In [ ]:
model = PPO(
    policy=policy,
    env=env,
    gamma=gamma,
    gae_lambda=gae_lambda,
    clip_range=clip_range,
    batch_size=batch_size)

if load_model:
    model.set_parameters(loaded_model_path)

In [ ]:
total_timesteps = 200_000_000

In [ ]:
saved_model_number = 20
save_freq = total_timesteps // (saved_model_number * num_threads)
log_interval = total_timesteps // 10

In [ ]:
checkpoint_callback = CheckpointCallback(
    save_freq=save_freq,
    save_path=save_path,
    name_prefix=model_filename_prefix)

In [ ]:
model.learn(
    total_timesteps=total_timesteps,
    log_interval=log_interval,
    callback=checkpoint_callback)

In [ ]:
model.save(f"{save_path}/{model_filename_prefix}")